<a href="https://colab.research.google.com/github/brianmulyadi/Designing-Data-Products/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import requests
from datetime import datetime
from datetime import date

In [2]:
# connect to ProPublica API
headers={'X-API-Key':'Guj6tsYRzPYaWECtImyu0NVttV3CrKbmD4NfXqHv'}
url = 'https://api.propublica.org/congress/v1/117/bills/hr1.json'
response = requests.get(url, headers=headers)
print(response.status_code)


200


In [4]:
# get congress members data
member_url = 'https://api.propublica.org/congress/v1/117/house/members.json'
response = requests.get(member_url, headers=headers)
json = response.json()

# generate list and enter into dataframe
memberId = []
fullName= []
dateOfBirth = []
gender = []
party = []
leadershipRole = []
dwNominate = []
seniority = []
votesAgainstParty = []

foo = 1

for x in json['results'][0]['members']:
  memberId.append(x['id'])
  fullName.append(x['first_name'] + " " + x['last_name'])
  dateOfBirth.append(x['date_of_birth'])
  gender.append(x['gender'])
  party.append(x['party'])
  leadershipRole.append(x['leadership_role'])
  dwNominate.append(x['dw_nominate'])
  seniority.append(x['seniority'])
  votesAgainstParty.append(x.get('votes_against_party_pct'))

# convert dateofBirth to age

#

member_list = list(zip(memberId, fullName, dateOfBirth, gender, party, leadershipRole, dwNominate, seniority, votesAgainstParty))
memberDf = pd.DataFrame(member_list, columns=['memberId', 'fullName', 'dateOfBirth', 'gender', 'party', 'leadershipRole', 'dwNominate', 'seniority', 'votesAgainstParty'])
memberDf.head()

,memberId,fullName,dateOfBirth,gender,party,leadershipRole,dwNominate,seniority,votesAgainstParty
0,A000370,Alma Adams,1946-05-27,F,D,None,-0.465,10,0.87
1,A000055,Robert Aderholt,1965-07-22,M,R,None,0.380,26,3.65
2,A000371,Pete Aguilar,1979-06-19,M,D,House Democratic Caucus Vice Chair,-0.296,8,1.09
3,A000372,Rick Allen,1951-11-07,M,R,None,0.699,8,8.97
4,A000376,Colin Allred,1983-04-15,M,D,None,NaN,4,1.75


In [5]:
memberDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   memberId           454 non-null    object 
 1   fullName           454 non-null    object 
 2   dateOfBirth        454 non-null    object 
 3   gender             454 non-null    object 
 4   party              454 non-null    object 
 5   leadershipRole     74 non-null     object 
 6   dwNominate         295 non-null    float64
 7   seniority          454 non-null    object 
 8   votesAgainstParty  453 non-null    float64
dtypes: float64(2), object(7)
memory usage: 32.0+ KB


### Cleaning up data

In [6]:
# change dateOfBirth to age
def calculate_age(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

memberDf['age'] = memberDf['dateOfBirth'].apply(calculate_age)
del memberDf['dateOfBirth']

memberDf.head()

,memberId,fullName,gender,party,leadershipRole,dwNominate,seniority,votesAgainstParty,age
0,A000370,Alma Adams,F,D,None,-0.465,10,0.87,76
1,A000055,Robert Aderholt,M,R,None,0.380,26,3.65,57
2,A000371,Pete Aguilar,M,D,House Democratic Caucus Vice Chair,-0.296,8,1.09,43
3,A000372,Rick Allen,M,R,None,0.699,8,8.97,70
4,A000376,Colin Allred,M,D,None,NaN,4,1.75,39


In [19]:
# change leadershipRole to boolean
memberDf['leadershipRole'] = np.where(memberDf['leadershipRole'].isnull(), 0, 1)
memberDf.head()

,memberId,fullName,gender,party,leadershipRole,dwNominate,seniority,votesAgainstParty,age
0,A000370,Alma Adams,F,D,0,-0.465,10,0.87,76
1,A000055,Robert Aderholt,M,R,0,0.380,26,3.65,57
2,A000371,Pete Aguilar,M,D,1,-0.296,8,1.09,43
3,A000372,Rick Allen,M,R,0,0.699,8,8.97,70
4,A000376,Colin Allred,M,D,0,NaN,4,1.75,39


In [50]:
# generate list and enter into dataframe (6353 1st session)
billId = []
billTitle= []
policyArea = []
introduceDate = []
memberId = []
sponsorParty = []
sponsorGender = []
sponsorBill = []
sponsorAgainst = []
cosponsors = []
cosponsorsDem = []
cosponsorsRep = []
billStatus = []

for x in range(4000, 6353):
  url = 'https://api.propublica.org/congress/v1/117/bills/hr' + str(x) + '.json'
  response = requests.get(url, headers=headers)
  json = response.json()
  billId.append(json['results'][0]['bill_slug'])
  billTitle.append(json['results'][0]['short_title'])
  policyArea.append(json['results'][0]['primary_subject'])
  introduceDate.append(json['results'][0]['introduced_date'])
  memberId.append(json['results'][0]['sponsor_id'])
  sponsorParty.append(json['results'][0]['sponsor_party'])
  cosponsors.append(json['results'][0]['cosponsors'])
  cosponsorsDem.append(json['results'][0]['cosponsors_by_party'].get('D'))
  cosponsorsRep.append(json['results'][0]['cosponsors_by_party'].get('R'))
  billStatus.append(json['results'][0]['house_passage'])
  

main_list = list(zip(billId, billTitle, policyArea, introduceDate, memberId, sponsorParty, cosponsors, cosponsorsDem, cosponsorsRep, billStatus))

df = pd.DataFrame(main_list, columns=['billId', 'billTitle', 'policyArea', 'introduceDate', 'memberId', 'sponsorParty', 'cosponsors', 'cosponsorsDem', 'cosponsorsRep', 'billStatus'])

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2353 entries, 0 to 2352
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   billId         2353 non-null   object 
 1   billTitle      2353 non-null   object 
 2   policyArea     2353 non-null   object 
 3   introduceDate  2353 non-null   object 
 4   memberId       2353 non-null   object 
 5   sponsorParty   2353 non-null   object 
 6   cosponsors     2353 non-null   int64  
 7   cosponsorsDem  1374 non-null   float64
 8   cosponsorsRep  1324 non-null   float64
 9   billStatus     145 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 184.0+ KB


In [52]:
# change billStatus to boolean

df['billStatus'] = np.where(df['billStatus'].isnull(), 0, 1)
df.head()

,billId,billTitle,policyArea,introduceDate,memberId,sponsorParty,cosponsors,cosponsorsDem,cosponsorsRep,billStatus
0,hr4000,Internet Application I.D. Act,Commerce,2021-06-17,K000378,R,0,NaN,NaN,0
1,hr4001,Online Retailer Product Origin Disclosure Act,Commerce,2021-06-17,K000378,R,0,NaN,NaN,0
2,hr4002,Expanding Access to Graduate Education Act of ...,Education,2021-06-17,K000382,D,4,2.0,2.0,0
3,hr4003,Work Without Worry Act,Social Welfare,2021-06-17,L000557,D,46,32.0,14.0,0
4,hr4004,No Student Exchanges with North Korean Schools...,Education,2021-06-17,L000589,R,5,NaN,5.0,0


In [53]:
# merge members dataframe to bills

mergeDf = pd.merge(df, memberDf, on='memberId', how='left')

In [54]:
mergeDf.tail()

,billId,billTitle,policyArea,introduceDate,memberId,sponsorParty,cosponsors,cosponsorsDem,cosponsorsRep,billStatus,fullName,gender,party,leadershipRole,dwNominate,seniority,votesAgainstParty,age
2348,hr6348,Climate Readiness Act of 2021,Armed Forces and National Security,2021-12-27,N000191,D,0,NaN,NaN,0,Joe Neguse,M,D,0,NaN,4,0.54,38
2349,hr6349,Teachers Are Leaders Act,Education,2021-12-27,N000191,D,0,NaN,NaN,0,Joe Neguse,M,D,0,NaN,4,0.54,38
2350,hr6350,Rescuing Every Contributor from Unwanted Recur...,Government Operations and Politics,2021-12-27,N000191,D,0,NaN,NaN,0,Joe Neguse,M,D,0,NaN,4,0.54,38
2351,hr6351,Climate Solutions Act of 2021,Energy,2021-12-30,L000582,D,0,NaN,NaN,0,Ted Lieu,M,D,0,-0.375,8,0.77,53
2352,hr6352,DRUGS Act,Health,2021-12-30,M001180,R,5,5.0,NaN,0,David McKinley,M,R,0,0.301,12,7.38,75


In [21]:
# import member data
from google.colab import files
memberDf.to_csv('members.csv')
files.download('members.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
# import bill data
from google.colab import files
mergeDf.to_csv('4000-6352.csv')
files.download('4000-6352.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>